In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from IPython.core.display import display, HTML
from IPython.lib.pretty import pprint
from itertools import chain

In [3]:
from mockdown.view import View

In [4]:
sample_view = View("p", (0, 0, 100, 100), children=[
    View("a1", (10, 10, 45, 90), children=[
        View("b11", (20, 20, 35, 45)),
        View("b12", (20, 55, 35, 80))
    ]),
    View("a2", (55, 10, 90, 90), children=[
        View("b21", (65, 20, 80, 45)),
        View("b22", (65, 55, 80, 80))
    ])
])

In [5]:
html_str = str(sample_view.to_display_html(scale=2))
display(HTML(html_str))

In [6]:
from mockdown.visibility import visible_pairs

edge_pairs = visible_pairs(sample_view, deep=True)
anchor_pairs = [(e1.anchor, e2.anchor) for (e1, e2) in edge_pairs]

In [7]:
html_str = str(sample_view.to_display_html(edge_pairs, scale=2))
display(HTML(html_str))

In [10]:
from mockdown.datalog import constraint_pairs
constraint_pairs(sample_view, anchor_pairs)

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', '_i2', 'display', 'HTML', 'pprint', 'chain', '_i3', 'View', '_i4', 'sample_view', '_i5', 'html_str', '_i6', 'visible_pairs', 'edge_pairs', 'anchor_pairs', '_i7', '_i8', 'constraint_pairs', '_i9', '_i10'])
